Imports for the functions

In [7]:
import collections
import random
import numpy as np
import re

In [ ]:
#Philani Mpofu (1848751)
#Chloe Smith (1877342)
#Matthew Kruger (1669326)
#Jesse Bristow (1875955)

Open txt file that is saved in the same folder. Then read in reviews and assign entries starting with ones to positive reviews and those with -1 to negative reviews

In [8]:
with open('simple-food-reviews.txt') as reviewsDoc:
    reviews = [line.rstrip() for line in reviewsDoc]
    positiveReviews = reviews[0:8]
    negativeReviews = reviews[9:17]

Just a function so split he words and return a counter set for the words. The counter function creates a two-tuple where you have (the word : number of occurences)
It needs to be passed a split array otherwise if you give it just a string it breaks down to letters - be warned.

In [9]:
def counter_text(randomReview):
    word = randomReview.split(' ')
    return collections.Counter(word)

Generate 12 random numbers from 0-17 so that we can choose some reviews to use for training. Assign the positive reviews to random positives so that we can tell which reviews are positive, do the same for the random negative reviews. Finally choose the reviews which are not known whether positive or negative that we will test using. Store their actual value 1 or -1 in an array with the corresponding indices.

In [10]:
x = np.array(random.sample(range(0,17),12))
y = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

randomReview = np.empty((0))
randomPositives = np.empty((0))
randomNegatives = np.empty((0))
testReviews = np.empty((0))
testReviewsAns = np.empty((0))
posC = collections.Counter()
negC = collections.Counter()
numN = 0
for i in range(12):
    if x[i] < 9:
        word = reviews[x[i]].split('1 ')
        randomPositives = np.append(randomPositives, word[1])
        posC = posC + counter_text(word[1])
    elif x[i] > 8:
        word = reviews[x[i]].split('-1 ')
        randomNegatives = np.append(randomNegatives, word[1])
        negC = negC + counter_text(word[1])
for i in range(18):
    if i not in x:
        word = reviews[i].split(' ', 1)
        testReviews = np.append(testReviews, word[1])
        testReviewsAns = np.append(testReviewsAns, word[0])
#print(posC)
#print(negC)

Now that we have them seperated we need to create a collection of words so that we can find the conditional probabilities. So let us have a list of the words and then their respective frequency - this will make it easy to add words should neither category contain them later on.

In [11]:
allC = posC + negC
itemsAll = list(allC.items())
wordsAll, numberAll = [list(c) for c in zip(*itemsAll)]
itemsNeg = list(negC.items())
wordsNeg, numberNeg = [list(c) for c in zip(*itemsNeg)]
itemsPos = list(posC.items())
wordsPos, numberPos = [list(c) for c in zip(*itemsPos)]
print(itemsAll)
print(wordsPos)
#print(numberPos)
print(wordsAll)
#print(numberAll)
print(wordsNeg)
#print(numberNeg)
#print(itemsNeg)
#print(itemsPos)

[('we', 1), ('had', 1), ('a', 3), ('lovely', 3), ('meal', 1), ('my', 2), ('food', 5), ('tasted', 2), ('great', 4), ('i', 2), ('really', 1), ('enjoyed', 2), ('what', 1), ('restaurant', 5), ('the', 9), ('service', 3), ('was', 2), ('experience', 1), ('at', 1), ('and', 1), ('this', 2), ('is', 3), ('terrible', 1), ('avoid', 1), ('awful', 1)]
['we', 'had', 'a', 'lovely', 'meal', 'my', 'food', 'tasted', 'great', 'i', 'really', 'enjoyed', 'what', 'restaurant', 'the', 'service', 'was', 'experience', 'at', 'and', 'this', 'is']
['we', 'had', 'a', 'lovely', 'meal', 'my', 'food', 'tasted', 'great', 'i', 'really', 'enjoyed', 'what', 'restaurant', 'the', 'service', 'was', 'experience', 'at', 'and', 'this', 'is', 'terrible', 'avoid', 'awful']
['the', 'service', 'is', 'terrible', 'avoid', 'this', 'restaurant', 'food', 'tasted', 'awful']


Calculate the probabilities of each probPos and probNeg. Could easily be done in one for loop. So if want to improve then just do it in one for loop instead. This should conclude the training portion since we have the required probabilities. I chose to do this function here so that we can use it in the question that does not do laplacian smoothing. It can be used easily then.
If we were going to select a piece of data that would confuse it then all we need to do is from the random set of reviews use the set function and then use the disjoint of the random review to check if it is not contained - since any element part of the random review that our model has not seen would give a 0 probability for both p(1) and p(-1) for the reviews actual value.

In [12]:
probPos = list()
probNeg = list()
size = len(wordsAll)
for i in range(size):
    try:
        index = wordsPos.index(wordsAll[i])
        if index >= 0:
            probPos.append(numberPos[index]/numberAll[i])
    except:
        probPos.append(0)
for i in range(size):
    try:
        index = wordsNeg.index(wordsAll[i])
        if index >= 0:
            probNeg.append(numberNeg[index]/numberAll[i])
    except:
        probNeg.append(0)

        
print(probPos)
print(probNeg)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.7777777777777778, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 0.5, 0.6666666666666666, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0.2, 0.5, 0, 0, 0, 0, 0, 0.2, 0.2222222222222222, 0.3333333333333333, 0, 0, 0, 0, 0.5, 0.3333333333333333, 1.0, 1.0, 1.0]


                                                                        ##CURENTLY NOT WORKING##

Now we need to take a sentence and calculate its most likely category, something is wrong here, definitely check to see what it is however feel free to check to see if the rest is working if you think I might have made a mistake. I tried to do a laplacian smoothing here where i checked to see if the word was contained in all before seen words and then to add an instance to each. It might be easier in this scenario to do a 'on the fly' check of each word, its index, number of times it occurs (since the list zip is in order) then we can always readily update however this will lead to slower computation speeds since we will be calculating that probability always. It might be easier for us though. Do what you feel would be easier or faster to complete the lab.

In [ ]:
def calcLikely(review, probPos, probNeg, wordsAll, wordsNeg, wordsPos, numberNeg, numberPos, numberAll, numP, numN):
   # print(review)
    temp1 = counter_text(review)
    temp2 = list(temp1.items())
   # print(temp2)
    testAll = collections.Counter(temp2)
  #  print(testAll)
    reviewWords, reviewNumbers = [list(c) for c in zip(*testAll)]
   # print(reviewWords)
    reviewLength = len(reviewWords)
    tempPos = float(1)
    tempNeg = float(1)
    for i in range(reviewLength):
        tempWord = reviewWords[i]
        print(i)
        print(tempPos)
        print(tempNeg)
        if tempWord in wordsAll:
            
            if tempWord in wordsNeg:
                index1 = wordsNeg.index(reviewWords[i])
                index2 = wordsAll.index(reviewWords[i])
                tempNeg = tempNeg * (numberNeg[index1]/numberAll[index2])
                                     
            if tempWord not in wordsNeg:
                                     
                index1 = wordsAll.index(reviewWords[i])
                tempNeg = tempNeg * (1 - probNeg[index1])
                                     
            if tempWord in wordsPos:
                index1 = wordsPos.index(reviewWords[i])
                index2 = wordsAll.index(reviewWords[i])
                tempPos = tempPos * (numberPos[index1]/numberAll[index2])
                                     
            if tempWord not in wordsPos:
                index1 = wordsAll.index(reviewWords[i])
                tempPos = tempPos * (1 - probPos[index1])
                
        else:
            wordsAll.append(reviewWords[i])
            wordsNeg.append(reviewWords[i])
            wordsPos.append(reviewWords[i])
            numberAll.append(1)
            numberNeg.append(1)
            numberPos.append(1)
            
    print(tempPos)
    print(tempNeg)
    tot = numP + numN
   #pos = float((tempPos*(numP/tot))/(tempPos*(numP/tot)+ tempNeg(numN/tot)))
   # neg = float((tempNeg*(numN/tot))/(tempNeg*(numN/tot)+ tempPos(numP/tot)))    
    propPos = numP/tot
    propNeg = numN/tot
    posNumer = tempPos * propPos
    negNum = tempNeg * propNeg
    posDenom = negNum + posNumer
    pos = posNumer/posDenom
    neg = 1 - pos
    
    
    probability = [pos, neg]
    return(probability)

Just a quick run through of all the random reviews so that we can calculate the probability of each as required. Shows the probability of each and then what the actual value was.

In [ ]:
testReviewsSize = len(testReviews)
numberOfPositives = len(randomPositives)
numberOfNegatives = len(randomNegatives)
#print(numberOfPositives)
#print(numberOfNegatives)
for i in range(testReviewsSize):
    probabilities = np.array(calcLikely(testReviews[i], probPos, probNeg, wordsAll, wordsNeg, wordsPos, numberNeg, numberPos, numberAll, numberOfPositives, numberOfNegatives))
    print("probability of 1 is " + str(probabilities[0]))
    print("probability of -1 is " + str(probabilities[1]))
    print("Answer is actually " + str(testReviewsAns[i]))
    #print(np.argmax(probabilities))
    
print("finished")